In [24]:
rm -rf ATML-PA-4

In [25]:
!git clone -b test http://github.com/Zapy67/ATML-PA-4 

Cloning into 'ATML-PA-4'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 158 (delta 75), reused 72 (delta 64), pack-reused 74 (from 1)
Receiving objects: 100% (158/158), 389.90 KiB | 13.00 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [27]:
!git pull  http://github.com/Zapy67/ATML-PA-4 -b test


fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [28]:
%cd ATML-PA-4

/kaggle/working/ATML-PA-4


In [29]:
!ls

'ATML - PA 4.pdf'   fedprox.ipynb       LICENSE       task2.ipynb
 fedgh.ipynb	    fedsam.ipynb        README.md     task3.ipynb
 fed_lib	    fedscaffold.ipynb   task1.ipynb   task4.ipynb


#  FedSAM on heterogeneous domains

In [32]:
import torch
import torch.nn as nn

In [38]:
import fed_lib
import importlib
importlib.reload(fed_lib)


<module 'fed_lib' from '/kaggle/working/ATML-PA-4/fed_lib/__init__.py'>

In [16]:
!pip install fedlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 17.7 MB/s eta 0:0

In [39]:
from fed_lib import fed_methods, fed_model, utils
importlib.reload(utils)
importlib.reload(fed_methods)
importlib.reload(fed_model)

<module 'fed_lib.fed_model' from '/kaggle/working/ATML-PA-4/fed_lib/fed_model.py'>

In [40]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
trainset, testset = utils.get_cifar10()

In [24]:
client_loaders = utils.get_heterogenous_domains(trainset, clients=5, balance=False, min_require_size=20, alpha=2, verbose=True)

In [26]:
for idx, loader in enumerate(client_loaders):
    utils.calculate_label_skew(loader.dataset)

Label Distribution (P(Y)): [0.039 0.197 0.102 0.121 0.082 0.167 0.02  0.031 0.115 0.126]
Entropy of Label Distribution H(P(Y)): 2.0409007076773857
Normalized Entropy of Label Distribution: 0.8863519154567302

Label Distribution (P(Y)): [0.082 0.079 0.135 0.053 0.128 0.1   0.034 0.144 0.244]
Entropy of Label Distribution H(P(Y)): 1.9797306458058213
Normalized Entropy of Label Distribution: 0.9010142459838699

Label Distribution (P(Y)): [0.042 0.035 0.101 0.016 0.078 0.123 0.207 0.043 0.072 0.283]
Entropy of Label Distribution H(P(Y)): 1.9221358831956668
Normalized Entropy of Label Distribution: 0.8347730075401114

Label Distribution (P(Y)): [0.112 0.132 0.102 0.209 0.087 0.025 0.151 0.182]
Entropy of Label Distribution H(P(Y)): 1.8967389565875812
Normalized Entropy of Label Distribution: 0.9121386288432701

Label Distribution (P(Y)): [0.242 0.035 0.055 0.092 0.128 0.076 0.104 0.105 0.063 0.1  ]
Entropy of Label Distribution H(P(Y)): 2.0809519365200044
Normalized Entropy of Label Distrib

In [27]:
client_weights = [len(client.dataset.indices)/len(trainset.dataset) for client in client_loaders]
domains = None

In [28]:
client_weights

[0.22526, 0.20142, 0.1886, 0.20208, 0.18264]

In [41]:
def fed_avg_exp(partition:str = 'dirichlet', alpha:float = 0.1, num_clients:int = 5, fed_method_class = None, **kwargs):
        fed = fed_model.Federation(
                num_clients=num_clients, federate_method=None, partition=partition, domains=None, alpha=alpha, device = device, batch_size=32, pin_memory=False, num_workers=2)
        
        client_loaders = fed.client_dataloaders
        client_weights = [len(client.dataset.indices)/len(trainset) for client in client_loaders]
        method_instance = None

        # if fed_method_class == fed_methods.FedSAM:
        #         rho = kwargs.get('rho', 0.05) # Safer access with default
        #         method_instance = fed_method_class(sam_rho=rho, 
        #                 num_local_steps=int(len(trainset)/num_clients), 
        #                 client_aggregation_weights=client_weights)
        
        # elif fed_method_class == fed_methods.FedGH:
        method_instance = fed_methods.FedGH(num_local_steps=int(len(trainset)/num_clients), 
                        client_aggregation_weights=client_weights)
        # else:
        #         raise Exception()
        

        fed.set_method(method_instance)
        fed.train(25, lr=1e-3, verbose=False, momentum=0, weight_decay=0)


In [ ]:
alphas = [0.1, 0.5, 1, 5, 10, 50, 100]
for alpha in alphas:
    fed_avg_exp(num_clients = 5, alpha=alpha)


--- Round 1/25 ---
Training Clients
Training Client 1/5
Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 1:


FedSAM  | Test Loss: 161.6193, Test Acc: 10.09%

--- Round 2/25 ---
Training Clients
Training Client 1/5


Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 2:


FedSAM  | Test Loss: 142.2099, Test Acc: 20.16%

--- Round 3/25 ---
Training Clients
Training Client 1/5
Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 3:


FedSAM  | Test Loss: 122.6762, Test Acc: 29.86%

--- Round 4/25 ---
Training Clients
Training Client 1/5


...

FedSAM  | Test Loss: 34.4928, Test Acc: 81.63%

--- Round 24/25 ---
Training Clients
Training Client 1/5
Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 24:


FedSAM  | Test Loss: 34.2206, Test Acc: 81.98%

--- Round 25/25 ---
Training Clients
Training Client 1/5


Training Client 2/5
Training Client 3/5
Training Client 4/5
Training Client 5/5
Training Server
Evaluate on round 25:


FedSAM  | Test Loss: 33.9069, Test Acc: 82.19%
Training Complete!


FedSAM  | Test Loss: 33.9069, Test Acc: 82.19%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                5.187540e+00 6.388851e-01
backbone.2.block.0.weight                4.885939e+00 6.113057e-01
backbone.1.block.0.weight                4.325180e+00 7.034668e-01
backbone.1.block.3.weight                4.235101e+00 6.932738e-01
backbone.0.block.3.weight                4.172994e+00 7.846729e-01
Overall Statistics:
Total L2 difference:        1.177553e+01
Total parameters:           551,466


In [ ]:

fed_avg_exp(num_clients=5, alpha=0.1 ,fed_method = fed_methods.FedSAM, rho=1e-2)


--- Round 1/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.640418e+00
Client Drift Std: 2.851188e-01
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :2.249981e+00
Client Number: 0.000000e+00 Drift :2.452281e+00
Client Number: 4.000000e+00 Drift :2.608269e+00
Client Number: 3.000000e+00 Drift :2.821078e+00
Client Number: 1.000000e+00 Drift :3.070482e+00

Evaluate on round 1:


FedSAM  | Test Loss: 143.1869, Test Acc: 13.67%

--- Round 2/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 2.192140e+00
Client Drift Std: 2.932173e-01
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.817235e+00
Client Number: 0.000000e+00 Drift :1.998955e+00
Client Number: 4.000000e+00 Drift :2.119690e+00
Client Number: 3.000000e+00 Drift :2.368163e+00
Client Number: 1.000000e+00 Drift :2.656656e+00

Evaluate on round 2:


FedSAM  | Test Loss: 138.1331, Test Acc: 19.85%

--- Round 3/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.027160e+00
Client Drift Std: 2.609505e-01
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.665576e+00
Client Number: 0.000000e+00 Drift :1.826709e+00
Client Number: 4.000000e+00 Drift :2.048923e+00
Client Number: 3.000000e+00 Drift :2.189754e+00
Client Number: 1.000000e+00 Drift :2.404840e+00

Evaluate on round 3:


FedSAM  | Test Loss: 119.9482, Test Acc: 30.08%

--- Round 4/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.916054e+00
Client Drift Std: 2.498536e-01
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.603711e+00
Client Number: 0.000000e+00 Drift :1.697696e+00
Client Number: 4.000000e+00 Drift :1.913846e+00
Client Number: 3.000000e+00 Drift :2.070941e+00
Client Number: 1.000000e+00 Drift :2.294076e+00

Evaluate on round 4:


...

Training Server

Client Drift Stats:
Client Drift Mean: 1.705379e+00
Client Drift Std: 2.657289e-01
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.451294e+00
Client Number: 0.000000e+00 Drift :1.467267e+00
Client Number: 4.000000e+00 Drift :1.647520e+00
Client Number: 3.000000e+00 Drift :1.783972e+00
Client Number: 1.000000e+00 Drift :2.176843e+00

Evaluate on round 23:


FedSAM  | Test Loss: 64.9543, Test Acc: 62.50%

--- Round 24/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.703067e+00
Client Drift Std: 2.737456e-01
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.412575e+00
Client Number: 0.000000e+00 Drift :1.492325e+00
Client Number: 4.000000e+00 Drift :1.654850e+00
Client Number: 3.000000e+00 Drift :1.762604e+00
Client Number: 1.000000e+00 Drift :2.192981e+00

Evaluate on round 24:


FedSAM  | Test Loss: 61.1804, Test Acc: 64.17%

--- Round 25/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.702548e+00
Client Drift Std: 2.868108e-01
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.381351e+00
Client Number: 0.000000e+00 Drift :1.507082e+00
Client Number: 4.000000e+00 Drift :1.658156e+00
Client Number: 3.000000e+00 Drift :1.747953e+00
Client Number: 1.000000e+00 Drift :2.218198e+00

Evaluate on round 25:


FedSAM  | Test Loss: 59.1381, Test Acc: 66.39%
Training Complete!


FedSAM  | Test Loss: 59.1381, Test Acc: 66.39%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                3.660266e+00 4.975188e-01
backbone.2.block.0.weight                3.386585e+00 4.670861e-01
backbone.0.block.3.weight                3.183172e+00 6.952555e-01
backbone.1.block.0.weight                3.124298e+00 5.695951e-01
backbone.1.block.3.weight                3.104541e+00 5.704601e-01
Overall Statistics:
Total L2 difference:        8.487432e+00
Total parameters:           551,466


In [ ]:
fed_avg_exp(num_clients=5, alpha=0.5 ,fed_method = fed_methods.FedSAM,rho=1e-2)


--- Round 1/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.959466e+00
Client Drift Std: 1.398537e-01
Top 5 Clients with Greatest Drift
Client Number: 4.000000e+00 Drift :2.785179e+00
Client Number: 2.000000e+00 Drift :2.896677e+00
Client Number: 3.000000e+00 Drift :2.922696e+00
Client Number: 0.000000e+00 Drift :2.985420e+00
Client Number: 1.000000e+00 Drift :3.207360e+00

Evaluate on round 1:


FedSAM  | Test Loss: 149.2718, Test Acc: 12.49%

--- Round 2/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 2.428335e+00
Client Drift Std: 1.559182e-01
Top 5 Clients with Greatest Drift
Client Number: 4.000000e+00 Drift :2.203681e+00
Client Number: 2.000000e+00 Drift :2.318124e+00
Client Number: 3.000000e+00 Drift :2.479993e+00
Client Number: 0.000000e+00 Drift :2.480033e+00
Client Number: 1.000000e+00 Drift :2.659842e+00

Evaluate on round 2:


FedSAM  | Test Loss: 102.5180, Test Acc: 40.77%

--- Round 3/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.198995e+00
Client Drift Std: 1.424067e-01
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :2.030622e+00
Client Number: 4.000000e+00 Drift :2.056904e+00
Client Number: 0.000000e+00 Drift :2.198401e+00
Client Number: 3.000000e+00 Drift :2.307191e+00
Client Number: 1.000000e+00 Drift :2.401856e+00

Evaluate on round 3:


FedSAM  | Test Loss: 87.0607, Test Acc: 50.50%

--- Round 4/25 ---
Training Clients


...

Training Server

Client Drift Stats:
Client Drift Mean: 1.927400e+00
Client Drift Std: 9.670569e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.816599e+00
Client Number: 4.000000e+00 Drift :1.867311e+00
Client Number: 0.000000e+00 Drift :1.913099e+00
Client Number: 3.000000e+00 Drift :1.937774e+00
Client Number: 1.000000e+00 Drift :2.102215e+00

Evaluate on round 23:


FedSAM  | Test Loss: 41.5856, Test Acc: 77.47%

--- Round 24/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.928185e+00
Client Drift Std: 1.165738e-01
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.788564e+00
Client Number: 4.000000e+00 Drift :1.852630e+00
Client Number: 3.000000e+00 Drift :1.913625e+00
Client Number: 0.000000e+00 Drift :1.953208e+00
Client Number: 1.000000e+00 Drift :2.132896e+00

Evaluate on round 24:


FedSAM  | Test Loss: 41.2957, Test Acc: 76.67%

--- Round 25/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.916600e+00
Client Drift Std: 1.128726e-01
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.779531e+00
Client Number: 4.000000e+00 Drift :1.829361e+00
Client Number: 3.000000e+00 Drift :1.923874e+00
Client Number: 0.000000e+00 Drift :1.942355e+00
Client Number: 1.000000e+00 Drift :2.107876e+00

Evaluate on round 25:


FedSAM  | Test Loss: 40.7090, Test Acc: 77.56%
Training Complete!


FedSAM  | Test Loss: 40.7090, Test Acc: 77.56%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                4.272909e+00 5.570967e-01
backbone.2.block.0.weight                3.929978e+00 5.245911e-01
backbone.1.block.0.weight                3.598225e+00 6.237720e-01
backbone.0.block.3.weight                3.560002e+00 7.263763e-01
backbone.1.block.3.weight                3.529924e+00 6.214730e-01
Overall Statistics:
Total L2 difference:        9.771924e+00
Total parameters:           551,466


In [ ]:
fed_avg_exp(num_clients=5, alpha=1 ,fed_method = fed_methods.FedSAM,rho=1e-2)


--- Round 1/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 3.220318e+00
Client Drift Std: 8.632428e-02
Top 5 Clients with Greatest Drift
Client Number: 0.000000e+00 Drift :3.109359e+00
Client Number: 2.000000e+00 Drift :3.133012e+00
Client Number: 4.000000e+00 Drift :3.233909e+00
Client Number: 3.000000e+00 Drift :3.306005e+00
Client Number: 1.000000e+00 Drift :3.319303e+00

Evaluate on round 1:


FedSAM  | Test Loss: 141.0190, Test Acc: 22.57%

--- Round 2/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 2.596864e+00
Client Drift Std: 6.313129e-02
Top 5 Clients with Greatest Drift
Client Number: 0.000000e+00 Drift :2.495336e+00
Client Number: 1.000000e+00 Drift :2.575472e+00
Client Number: 4.000000e+00 Drift :2.586119e+00
Client Number: 2.000000e+00 Drift :2.656150e+00
Client Number: 3.000000e+00 Drift :2.671242e+00

Evaluate on round 2:


FedSAM  | Test Loss: 95.1761, Test Acc: 45.47%

--- Round 3/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 2.297185e+00
Client Drift Std: 2.457123e-02
Top 5 Clients with Greatest Drift
Client Number: 4.000000e+00 Drift :2.268756e+00
Client Number: 2.000000e+00 Drift :2.270074e+00
Client Number: 1.000000e+00 Drift :2.299304e+00
Client Number: 3.000000e+00 Drift :2.319202e+00
Client Number: 0.000000e+00 Drift :2.328590e+00

Evaluate on round 3:


...

FedSAM  | Test Loss: 37.7494, Test Acc: 79.34%

--- Round 24/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.115402e+00
Client Drift Std: 1.002026e-01
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.934633e+00
Client Number: 4.000000e+00 Drift :2.095926e+00
Client Number: 1.000000e+00 Drift :2.149457e+00
Client Number: 3.000000e+00 Drift :2.165509e+00
Client Number: 0.000000e+00 Drift :2.231486e+00

Evaluate on round 24:


FedSAM  | Test Loss: 38.6734, Test Acc: 79.01%

--- Round 25/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 2.112820e+00
Client Drift Std: 9.928226e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.934983e+00
Client Number: 1.000000e+00 Drift :2.121172e+00
Client Number: 3.000000e+00 Drift :2.131473e+00
Client Number: 4.000000e+00 Drift :2.134082e+00
Client Number: 0.000000e+00 Drift :2.242392e+00

Evaluate on round 25:


FedSAM  | Test Loss: 36.7279, Test Acc: 80.13%
Training Complete!


FedSAM  | Test Loss: 36.7279, Test Acc: 80.13%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                4.790715e+00 6.030622e-01
backbone.2.block.0.weight                4.491172e+00 5.768811e-01
backbone.1.block.0.weight                4.008567e+00 6.698293e-01
backbone.0.block.3.weight                3.948005e+00 7.613248e-01
backbone.1.block.3.weight                3.915238e+00 6.600068e-01
Overall Statistics:
Total L2 difference:        1.092812e+01
Total parameters:           551,466


In [ ]:
fed_avg_exp(num_clients=5, alpha=5 ,fed_method = fed_methods.FedSAM,rho=1e-2)


--- Round 1/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 3.363762e+00
Client Drift Std: 5.873007e-02
Top 5 Clients with Greatest Drift
Client Number: 4.000000e+00 Drift :3.298209e+00
Client Number: 0.000000e+00 Drift :3.304756e+00
Client Number: 1.000000e+00 Drift :3.359484e+00
Client Number: 3.000000e+00 Drift :3.404204e+00
Client Number: 2.000000e+00 Drift :3.452155e+00

Evaluate on round 1:


FedSAM  | Test Loss: 142.2063, Test Acc: 19.76%

--- Round 2/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.630023e+00
Client Drift Std: 3.907416e-02
Top 5 Clients with Greatest Drift
Client Number: 4.000000e+00 Drift :2.552179e+00
Client Number: 3.000000e+00 Drift :2.643492e+00
Client Number: 0.000000e+00 Drift :2.649128e+00
Client Number: 1.000000e+00 Drift :2.651419e+00
Client Number: 2.000000e+00 Drift :2.653898e+00

Evaluate on round 2:


FedSAM  | Test Loss: 93.0842, Test Acc: 47.16%

--- Round 3/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.406103e+00
Client Drift Std: 6.245294e-02
Top 5 Clients with Greatest Drift
Client Number: 4.000000e+00 Drift :2.335690e+00
Client Number: 1.000000e+00 Drift :2.361149e+00
Client Number: 3.000000e+00 Drift :2.376684e+00
Client Number: 0.000000e+00 Drift :2.454245e+00
Client Number: 2.000000e+00 Drift :2.502746e+00

Evaluate on round 3:


FedSAM  | Test Loss: 77.5215, Test Acc: 56.65%

--- Round 4/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.315028e+00
Client Drift Std: 4.192859e-02
Top 5 Clients with Greatest Drift
Client Number: 4.000000e+00 Drift :2.251974e+00
Client Number: 1.000000e+00 Drift :2.279537e+00
Client Number: 3.000000e+00 Drift :2.334843e+00
Client Number: 0.000000e+00 Drift :2.348790e+00
Client Number: 2.000000e+00 Drift :2.359995e+00

Evaluate on round 4:


FedSAM  | Test Loss: 70.2261, Test Acc: 60.83%

--- Round 5/25 ---
Training Clients


...

FedSAM  | Test Loss: 35.7170, Test Acc: 80.47%

--- Round 24/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 2.178657e+00
Client Drift Std: 4.062152e-02
Top 5 Clients with Greatest Drift
Client Number: 4.000000e+00 Drift :2.110495e+00
Client Number: 3.000000e+00 Drift :2.154709e+00
Client Number: 1.000000e+00 Drift :2.197886e+00
Client Number: 0.000000e+00 Drift :2.213475e+00
Client Number: 2.000000e+00 Drift :2.216721e+00

Evaluate on round 24:


FedSAM  | Test Loss: 35.1191, Test Acc: 80.83%

--- Round 25/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.189350e+00
Client Drift Std: 4.505251e-02
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :2.131722e+00
Client Number: 4.000000e+00 Drift :2.140189e+00
Client Number: 1.000000e+00 Drift :2.206602e+00
Client Number: 0.000000e+00 Drift :2.226886e+00
Client Number: 2.000000e+00 Drift :2.241353e+00

Evaluate on round 25:


FedSAM  | Test Loss: 35.5223, Test Acc: 80.87%
Training Complete!


FedSAM  | Test Loss: 35.5223, Test Acc: 80.87%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                5.108386e+00 6.309958e-01
backbone.2.block.0.weight                4.762250e+00 6.020836e-01
backbone.1.block.0.weight                4.168437e+00 6.838788e-01
backbone.1.block.3.weight                4.112824e+00 6.823302e-01
backbone.0.block.3.weight                4.022152e+00 7.709921e-01
Overall Statistics:
Total L2 difference:        1.145895e+01
Total parameters:           551,466


In [ ]:
fed_avg_exp(num_clients=5, alpha=10 ,fed_method = fed_methods.FedSAM,rho=1e-2)


--- Round 1/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 3.314403e+00
Client Drift Std: 5.661286e-02
Top 5 Clients with Greatest Drift
Client Number: 0.000000e+00 Drift :3.229836e+00
Client Number: 1.000000e+00 Drift :3.284657e+00
Client Number: 2.000000e+00 Drift :3.306220e+00
Client Number: 3.000000e+00 Drift :3.361184e+00
Client Number: 4.000000e+00 Drift :3.390121e+00

Evaluate on round 1:


FedSAM  | Test Loss: 142.4844, Test Acc: 18.07%

--- Round 2/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.485270e+00
Client Drift Std: 6.111128e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :2.372319e+00
Client Number: 3.000000e+00 Drift :2.476198e+00
Client Number: 0.000000e+00 Drift :2.503313e+00
Client Number: 4.000000e+00 Drift :2.530149e+00
Client Number: 1.000000e+00 Drift :2.544372e+00

Evaluate on round 2:


FedSAM  | Test Loss: 91.0771, Test Acc: 48.28%

--- Round 3/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 2.325839e+00
Client Drift Std: 5.452533e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :2.252088e+00
Client Number: 0.000000e+00 Drift :2.268651e+00
Client Number: 4.000000e+00 Drift :2.352770e+00
Client Number: 3.000000e+00 Drift :2.374568e+00
Client Number: 1.000000e+00 Drift :2.381118e+00

Evaluate on round 3:


FedSAM  | Test Loss: 77.5229, Test Acc: 56.69%

--- Round 4/25 ---
Training Clients


...

Training Server

Client Drift Stats:
Client Drift Mean: 2.184824e+00
Client Drift Std: 3.795763e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :2.128665e+00
Client Number: 0.000000e+00 Drift :2.178738e+00
Client Number: 4.000000e+00 Drift :2.183377e+00
Client Number: 3.000000e+00 Drift :2.185183e+00
Client Number: 1.000000e+00 Drift :2.248155e+00

Evaluate on round 24:


FedSAM  | Test Loss: 36.0779, Test Acc: 80.81%

--- Round 25/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 2.203777e+00
Client Drift Std: 4.610092e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :2.116381e+00
Client Number: 4.000000e+00 Drift :2.209623e+00
Client Number: 3.000000e+00 Drift :2.210769e+00
Client Number: 0.000000e+00 Drift :2.233118e+00
Client Number: 1.000000e+00 Drift :2.248992e+00

Evaluate on round 25:


FedSAM  | Test Loss: 35.0508, Test Acc: 81.36%
Training Complete!


FedSAM  | Test Loss: 35.0508, Test Acc: 81.36%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                5.168962e+00 6.365455e-01
backbone.2.block.0.weight                4.816781e+00 6.047602e-01
backbone.1.block.0.weight                4.183267e+00 6.867255e-01
backbone.1.block.3.weight                4.121691e+00 6.802217e-01
backbone.0.block.3.weight                4.120381e+00 7.750913e-01
Overall Statistics:
Total L2 difference:        1.163826e+01
Total parameters:           551,466


In [73]:
fed_avg_exp(num_clients=5, alpha=0.1 ,fed_method = fed_methods.FedGH)


--- Round 1/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.546877e+00
Client Drift Std: 4.712773e-01
Top 5 Clients with Greatest Drift
Client Number: 1.000000e+00 Drift :1.656885e+00
Client Number: 0.000000e+00 Drift :2.488780e+00
Client Number: 4.000000e+00 Drift :2.781016e+00
Client Number: 3.000000e+00 Drift :2.852046e+00
Client Number: 2.000000e+00 Drift :2.955657e+00

Evaluate on round 1:


FedSAM  | Test Loss: 153.4940, Test Acc: 10.14%

--- Round 2/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 2.046671e+00
Client Drift Std: 4.491879e-01
Top 5 Clients with Greatest Drift
Client Number: 1.000000e+00 Drift :1.174267e+00
Client Number: 0.000000e+00 Drift :2.153055e+00
Client Number: 3.000000e+00 Drift :2.159193e+00
Client Number: 4.000000e+00 Drift :2.301827e+00
Client Number: 2.000000e+00 Drift :2.445013e+00

Evaluate on round 2:


FedSAM  | Test Loss: 129.7018, Test Acc: 23.36%

--- Round 3/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.914376e+00
Client Drift Std: 3.982452e-01
Top 5 Clients with Greatest Drift
Client Number: 1.000000e+00 Drift :1.126653e+00
Client Number: 3.000000e+00 Drift :2.038261e+00
Client Number: 0.000000e+00 Drift :2.069674e+00
Client Number: 4.000000e+00 Drift :2.125543e+00
Client Number: 2.000000e+00 Drift :2.211751e+00

Evaluate on round 3:


FedSAM  | Test Loss: 114.2930, Test Acc: 35.16%

--- Round 4/25 ---
Training Clients


...

FedSAM  | Test Loss: 60.8575, Test Acc: 66.61%

--- Round 23/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.746097e+00
Client Drift Std: 3.933174e-01
Top 5 Clients with Greatest Drift
Client Number: 1.000000e+00 Drift :1.078381e+00
Client Number: 3.000000e+00 Drift :1.582333e+00
Client Number: 2.000000e+00 Drift :1.807662e+00
Client Number: 4.000000e+00 Drift :2.112973e+00
Client Number: 0.000000e+00 Drift :2.149137e+00

Evaluate on round 23:


FedSAM  | Test Loss: 56.3178, Test Acc: 69.32%

--- Round 24/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.752317e+00
Client Drift Std: 3.773346e-01
Top 5 Clients with Greatest Drift
Client Number: 1.000000e+00 Drift :1.108308e+00
Client Number: 3.000000e+00 Drift :1.622200e+00
Client Number: 2.000000e+00 Drift :1.778957e+00
Client Number: 4.000000e+00 Drift :2.117288e+00
Client Number: 0.000000e+00 Drift :2.134829e+00

Evaluate on round 24:


FedSAM  | Test Loss: 58.3385, Test Acc: 68.14%

--- Round 25/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.749526e+00
Client Drift Std: 3.828629e-01
Top 5 Clients with Greatest Drift
Client Number: 1.000000e+00 Drift :1.104014e+00
Client Number: 3.000000e+00 Drift :1.580553e+00
Client Number: 2.000000e+00 Drift :1.813756e+00
Client Number: 4.000000e+00 Drift :2.092148e+00
Client Number: 0.000000e+00 Drift :2.157161e+00

Evaluate on round 25:


FedSAM  | Test Loss: 57.4663, Test Acc: 68.80%
Training Complete!


FedSAM  | Test Loss: 57.4663, Test Acc: 68.80%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                3.821509e+00 5.144010e-01
backbone.2.block.0.weight                3.611791e+00 4.929875e-01
backbone.0.block.3.weight                3.347074e+00 7.128220e-01
backbone.1.block.0.weight                3.288770e+00 5.941299e-01
backbone.1.block.3.weight                3.119159e+00 5.668115e-01
Overall Statistics:
Total L2 difference:        8.915998e+00
Total parameters:           551,466


In [74]:
fed_avg_exp(num_clients=5, alpha=0.5 ,fed_method = fed_methods.FedGH)


--- Round 1/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.891659e+00
Client Drift Std: 1.344370e-01
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :2.695518e+00
Client Number: 1.000000e+00 Drift :2.845329e+00
Client Number: 0.000000e+00 Drift :2.855793e+00
Client Number: 4.000000e+00 Drift :2.961091e+00
Client Number: 2.000000e+00 Drift :3.100565e+00

Evaluate on round 1:


FedSAM  | Test Loss: 143.9430, Test Acc: 17.03%

--- Round 2/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 2.049286e+00
Client Drift Std: 1.729699e-01
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :1.777107e+00
Client Number: 1.000000e+00 Drift :1.976814e+00
Client Number: 0.000000e+00 Drift :2.026081e+00
Client Number: 4.000000e+00 Drift :2.201108e+00
Client Number: 2.000000e+00 Drift :2.265321e+00

Evaluate on round 2:


FedSAM  | Test Loss: 109.0722, Test Acc: 38.30%

--- Round 3/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.751215e+00
Client Drift Std: 1.392583e-01
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :1.500875e+00
Client Number: 0.000000e+00 Drift :1.733287e+00
Client Number: 1.000000e+00 Drift :1.776709e+00
Client Number: 4.000000e+00 Drift :1.828707e+00
Client Number: 2.000000e+00 Drift :1.916496e+00

Evaluate on round 3:


FedSAM  | Test Loss: 95.8613, Test Acc: 46.46%

--- Round 4/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.663317e+00
Client Drift Std: 1.280407e-01
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :1.417384e+00
Client Number: 1.000000e+00 Drift :1.685016e+00
Client Number: 0.000000e+00 Drift :1.693016e+00
Client Number: 4.000000e+00 Drift :1.735947e+00
Client Number: 2.000000e+00 Drift :1.785222e+00

Evaluate on round 4:


...

Training Server

Client Drift Stats:
Client Drift Mean: 1.491249e+00
Client Drift Std: 9.530382e-02
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :1.302493e+00
Client Number: 2.000000e+00 Drift :1.521535e+00
Client Number: 0.000000e+00 Drift :1.531518e+00
Client Number: 4.000000e+00 Drift :1.538942e+00
Client Number: 1.000000e+00 Drift :1.561759e+00

Evaluate on round 24:


FedSAM  | Test Loss: 49.2616, Test Acc: 73.50%

--- Round 25/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 1.483913e+00
Client Drift Std: 8.932033e-02
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :1.311014e+00
Client Number: 2.000000e+00 Drift :1.495544e+00
Client Number: 4.000000e+00 Drift :1.518546e+00
Client Number: 0.000000e+00 Drift :1.529291e+00
Client Number: 1.000000e+00 Drift :1.565170e+00

Evaluate on round 25:


FedSAM  | Test Loss: 49.8703, Test Acc: 72.45%
Training Complete!


FedSAM  | Test Loss: 49.8703, Test Acc: 72.45%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                4.308949e+00 5.493431e-01
backbone.2.block.5.weight                3.841138e+00 5.135173e-01
backbone.0.block.3.weight                3.833709e+00 7.447975e-01
backbone.1.block.0.weight                3.742681e+00 6.205353e-01
backbone.2.block.0.weight                3.708128e+00 4.900747e-01
Overall Statistics:
Total L2 difference:        1.066497e+01
Total parameters:           551,466


In [75]:
fed_avg_exp(num_clients=5, alpha=1 ,fed_method = fed_methods.FedGH)


--- Round 1/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.946372e+00
Client Drift Std: 1.964926e-01
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :2.628933e+00
Client Number: 0.000000e+00 Drift :2.850374e+00
Client Number: 4.000000e+00 Drift :2.958745e+00
Client Number: 1.000000e+00 Drift :3.111653e+00
Client Number: 2.000000e+00 Drift :3.182153e+00

Evaluate on round 1:


FedSAM  | Test Loss: 140.1678, Test Acc: 20.14%

--- Round 2/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.990850e+00
Client Drift Std: 1.884938e-01
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :1.635498e+00
Client Number: 0.000000e+00 Drift :2.002889e+00
Client Number: 4.000000e+00 Drift :2.022180e+00
Client Number: 1.000000e+00 Drift :2.118725e+00
Client Number: 2.000000e+00 Drift :2.174959e+00

Evaluate on round 2:


FedSAM  | Test Loss: 108.2625, Test Acc: 38.47%

--- Round 3/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 1.719126e+00
Client Drift Std: 1.890182e-01
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :1.349860e+00
Client Number: 0.000000e+00 Drift :1.766566e+00
Client Number: 4.000000e+00 Drift :1.784615e+00
Client Number: 1.000000e+00 Drift :1.809297e+00
Client Number: 2.000000e+00 Drift :1.885293e+00

Evaluate on round 3:


FedSAM  | Test Loss: 98.5384, Test Acc: 41.98%

--- Round 4/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.638461e+00
Client Drift Std: 1.956587e-01
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :1.256628e+00
Client Number: 0.000000e+00 Drift :1.685514e+00
Client Number: 4.000000e+00 Drift :1.695595e+00
Client Number: 1.000000e+00 Drift :1.749050e+00
Client Number: 2.000000e+00 Drift :1.805516e+00

Evaluate on round 4:


...

FedSAM  | Test Loss: 46.7244, Test Acc: 74.63%

--- Round 24/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 1.474363e+00
Client Drift Std: 2.011325e-01
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :1.078994e+00
Client Number: 2.000000e+00 Drift :1.509064e+00
Client Number: 4.000000e+00 Drift :1.571153e+00
Client Number: 0.000000e+00 Drift :1.589471e+00
Client Number: 1.000000e+00 Drift :1.623132e+00

Evaluate on round 24:


FedSAM  | Test Loss: 45.5788, Test Acc: 75.69%

--- Round 25/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.454882e+00
Client Drift Std: 2.028070e-01
Top 5 Clients with Greatest Drift
Client Number: 3.000000e+00 Drift :1.057559e+00
Client Number: 2.000000e+00 Drift :1.478995e+00
Client Number: 4.000000e+00 Drift :1.559229e+00
Client Number: 0.000000e+00 Drift :1.579993e+00
Client Number: 1.000000e+00 Drift :1.598636e+00

Evaluate on round 25:


FedSAM  | Test Loss: 44.7854, Test Acc: 76.08%
Training Complete!


FedSAM  | Test Loss: 44.7854, Test Acc: 76.08%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                4.427714e+00 5.612502e-01
backbone.2.block.5.weight                3.922455e+00 5.302773e-01
backbone.0.block.3.weight                3.855479e+00 7.512340e-01
backbone.2.block.0.weight                3.813832e+00 4.995223e-01
backbone.1.block.0.weight                3.717007e+00 6.224720e-01
Overall Statistics:
Total L2 difference:        1.083358e+01
Total parameters:           551,466


In [76]:
fed_avg_exp(num_clients=5, alpha=5 ,fed_method = fed_methods.FedGH)


--- Round 1/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 3.088203e+00
Client Drift Std: 7.461181e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :2.977834e+00
Client Number: 1.000000e+00 Drift :3.020029e+00
Client Number: 3.000000e+00 Drift :3.132350e+00
Client Number: 0.000000e+00 Drift :3.151793e+00
Client Number: 4.000000e+00 Drift :3.159005e+00

Evaluate on round 1:


FedSAM  | Test Loss: 143.8636, Test Acc: 13.25%

--- Round 2/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.011951e+00
Client Drift Std: 4.547781e-02
Top 5 Clients with Greatest Drift
Client Number: 1.000000e+00 Drift :1.941020e+00
Client Number: 2.000000e+00 Drift :1.978749e+00
Client Number: 3.000000e+00 Drift :2.033798e+00
Client Number: 0.000000e+00 Drift :2.039966e+00
Client Number: 4.000000e+00 Drift :2.066223e+00

Evaluate on round 2:


FedSAM  | Test Loss: 101.3089, Test Acc: 44.11%

--- Round 3/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 1.680748e+00
Client Drift Std: 4.689663e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.595773e+00
Client Number: 0.000000e+00 Drift :1.670014e+00
Client Number: 1.000000e+00 Drift :1.697222e+00
Client Number: 3.000000e+00 Drift :1.709185e+00
Client Number: 4.000000e+00 Drift :1.731546e+00

Evaluate on round 3:


FedSAM  | Test Loss: 90.4074, Test Acc: 49.63%

--- Round 4/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.635288e+00
Client Drift Std: 5.364645e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.533286e+00
Client Number: 1.000000e+00 Drift :1.637938e+00
Client Number: 0.000000e+00 Drift :1.647337e+00
Client Number: 3.000000e+00 Drift :1.675601e+00
Client Number: 4.000000e+00 Drift :1.682276e+00

Evaluate on round 4:


...

Training Server

Client Drift Stats:
Client Drift Mean: 1.474814e+00
Client Drift Std: 2.704188e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.431729e+00
Client Number: 3.000000e+00 Drift :1.459249e+00
Client Number: 1.000000e+00 Drift :1.482663e+00
Client Number: 0.000000e+00 Drift :1.490072e+00
Client Number: 4.000000e+00 Drift :1.510357e+00

Evaluate on round 24:


FedSAM  | Test Loss: 42.6410, Test Acc: 76.79%

--- Round 25/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 1.465182e+00
Client Drift Std: 2.387834e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.424146e+00
Client Number: 3.000000e+00 Drift :1.454577e+00
Client Number: 1.000000e+00 Drift :1.475964e+00
Client Number: 0.000000e+00 Drift :1.478415e+00
Client Number: 4.000000e+00 Drift :1.492806e+00

Evaluate on round 25:


FedSAM  | Test Loss: 41.8409, Test Acc: 77.05%
Training Complete!


FedSAM  | Test Loss: 41.8409, Test Acc: 77.05%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                4.708797e+00 5.849145e-01
backbone.0.block.3.weight                4.180692e+00 7.780704e-01
backbone.1.block.0.weight                4.100698e+00 6.591800e-01
backbone.2.block.0.weight                4.073315e+00 5.269514e-01
backbone.1.block.3.weight                3.946137e+00 6.486644e-01
Overall Statistics:
Total L2 difference:        1.160472e+01
Total parameters:           551,466


In [77]:
fed_avg_exp(num_clients=5, alpha=10 ,fed_method = fed_methods.FedGH)


--- Round 1/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 2.988503e+00
Client Drift Std: 7.047697e-02
Top 5 Clients with Greatest Drift
Client Number: 4.000000e+00 Drift :2.892718e+00
Client Number: 2.000000e+00 Drift :2.950715e+00
Client Number: 0.000000e+00 Drift :2.958730e+00
Client Number: 3.000000e+00 Drift :3.068655e+00
Client Number: 1.000000e+00 Drift :3.071698e+00

Evaluate on round 1:


FedSAM  | Test Loss: 143.7746, Test Acc: 19.27%

--- Round 2/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 1.862540e+00
Client Drift Std: 3.214144e-02
Top 5 Clients with Greatest Drift
Client Number: 0.000000e+00 Drift :1.823130e+00
Client Number: 2.000000e+00 Drift :1.837707e+00
Client Number: 1.000000e+00 Drift :1.851655e+00
Client Number: 3.000000e+00 Drift :1.894860e+00
Client Number: 4.000000e+00 Drift :1.905347e+00

Evaluate on round 2:


FedSAM  | Test Loss: 98.0678, Test Acc: 44.61%

--- Round 3/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.584801e+00
Client Drift Std: 4.861428e-02
Top 5 Clients with Greatest Drift
Client Number: 0.000000e+00 Drift :1.537137e+00
Client Number: 2.000000e+00 Drift :1.539006e+00
Client Number: 1.000000e+00 Drift :1.579563e+00
Client Number: 3.000000e+00 Drift :1.598441e+00
Client Number: 4.000000e+00 Drift :1.669856e+00

Evaluate on round 3:


FedSAM  | Test Loss: 88.8537, Test Acc: 51.38%

--- Round 4/25 ---
Training Clients


...

FedSAM  | Test Loss: 41.6248, Test Acc: 77.58%

--- Round 24/25 ---
Training Clients


Training Server

Client Drift Stats:
Client Drift Mean: 1.488361e+00
Client Drift Std: 1.277587e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.471821e+00
Client Number: 3.000000e+00 Drift :1.477818e+00
Client Number: 0.000000e+00 Drift :1.491103e+00
Client Number: 4.000000e+00 Drift :1.492545e+00
Client Number: 1.000000e+00 Drift :1.508519e+00

Evaluate on round 24:


FedSAM  | Test Loss: 40.9683, Test Acc: 77.95%

--- Round 25/25 ---
Training Clients
Training Server

Client Drift Stats:
Client Drift Mean: 1.487919e+00
Client Drift Std: 1.486236e-02
Top 5 Clients with Greatest Drift
Client Number: 2.000000e+00 Drift :1.477223e+00
Client Number: 0.000000e+00 Drift :1.479608e+00
Client Number: 3.000000e+00 Drift :1.482678e+00
Client Number: 1.000000e+00 Drift :1.482730e+00
Client Number: 4.000000e+00 Drift :1.517357e+00

Evaluate on round 25:


FedSAM  | Test Loss: 39.9957, Test Acc: 78.86%
Training Complete!


FedSAM  | Test Loss: 39.9957, Test Acc: 78.86%
Comparing Server vs Central

Top 5 layers with largest L2 differences:
Layer                                         L2 Diff     Rel Diff
backbone.2.block.3.weight                4.805043e+00 5.935362e-01
backbone.2.block.0.weight                4.255374e+00 5.422091e-01
backbone.0.block.3.weight                4.228393e+00 7.755181e-01
backbone.1.block.0.weight                4.160676e+00 6.632040e-01
backbone.1.block.3.weight                4.056984e+00 6.597667e-01
Overall Statistics:
Total L2 difference:        1.184296e+01
Total parameters:           551,466


In [42]:
fed_avg_exp(num_clients=5, alpha=5 ,fed_method=fed_methods.FedGH)


--- Round 1/25 ---
Training Clients
Training Server
Training Central


Evaluate on round 1:


FedGH   | Test Loss: 1597737160.0896, Test Acc: 12.06%
Central | Test Loss: 75.4503, Test Acc: 10.39%


AttributeError: 'NoneType' object has no attribute 'eval'